## Exercise 2: Rigid and perspective transformations in homogeneous coordinates


In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In these exercises we will assume a modern camera with completely square pixels. What are the skew parameters then?

- The skew parameters α and β, where α = 1 and β=0. 

### Mathematical exercises: Pinhole camera

- **Exercise 2.1**. Reuse the `box3` function from last week. Assume that f = 600, α = 1, β = 0, and δx = δy = 400. Given a traditional camera, what is the resolution in pixels?

For this camera the sensor has 2 × 400 = 800 pixels along each dimension i.e. a resolution of **800 × 800 pixels.**

$$ K = \begin{bmatrix} f & \beta f & \delta_x \\ 0 & \alpha f & \delta_y \\ 0 & 0 & 1 \end{bmatrix}$$


$$ K = \begin{bmatrix} 600 & 0 & 400 \\ 0 & 600 & 400 \\ 0 & 0 & 1 \end{bmatrix}$$

Also assume $R = I$, and $t = [0,.2,1.5]^T.$ Use `projectpoint` from last week, to project the points.

In [5]:
def box3d(n):
    comb=[(0.5,0.5),(0.5,-0.5),(-0.5,0.5),(-0.5,-0.5)]
    a=np.linspace(-0.5, 0.5, n)
    box=[0,0,0]

    for j in comb:
        for i in a:
            p=[i,j[0],j[1]]
            box=np.vstack([box, p])

    for j in comb:
        for i in a[1:-1]:
            p=[j[0],i,j[1]]
            box=np.vstack([box, p])

    for j in comb:
        for i in a[1:-1]:
            p=[j[0],j[1],i]
            box=np.vstack([box, p])

    for i in a:
        cross1=[i,0,0]   
        box=np.vstack([box, cross1])
    for i in a:
        cross2=[0,i,0]   
        box=np.vstack([box, cross2])
    for i in a:
        cross3=[0,0,i]   
        box=np.vstack([box, cross3])
      
    box=box[1:]

    return box


def projectpoints(K,Cam,Q):
    #projection matrix P
    P=K@Cam
    
    [m,n]=np.shape(Q)
    cnt=np.ones(m)
    Q_ext=np.column_stack((Q,cnt))
    
    Q_ext=np.transpose(Q_ext)
    
    projection=P@(Q_ext)
    
    qx=np.transpose([projection[0]/projection[2]])
    qy=np.transpose([projection[1]/projection[2]])
    
    project2D=np.column_stack((qx,qy))
    
    project3D=np.transpose(projection)
    
    return P,project2D, project3D

In [9]:
K=np.array([[600,0,400],[0,600,400],[0,0,1]]) #camera matrix
R=np.array([[1,0,0],[0,1,0],[0,0,1]]) #rotation
t=np.transpose([0,0.2,1.5]) #translation
Q=box3d(16) #n points in 3D to be projected into the camera

Cam=np.column_stack((R,t))

P,points2D,points3D=projectpoints(K,Cam,Q)
print('The projection matrix of the camera is:')
print(P)

The projection matrix of the camera is:
[[600.    0.  400.  600. ]
 [  0.  600.  400.  720. ]
 [  0.    0.    1.    1.5]]


Are all the points are captured by the image sensor? Where does the corner $P_1 = [−0.5, −0.5, −0.5]$ project to?

<font color='darkblue'> Some points have an y value greater than 800, and are not visible in the image, as they are outside the image sensor.
    
    
    

In [ ]:
#Function created to see the projected point of a 
def projectpoint(coord)


    return coord